In [ ]:
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
url_dave = 'https://davesmusicdatabase.blogspot.com/2014/08/top-100-songs-from-1980-to-1989.html'
headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

response = requests.get(url_dave, headers=headers)

response.status_code

In [ ]:
soup = BeautifulSoup(response.content)
soup


In [ ]:
#Obtengo un set de los tags presentes en el tag 'p'
tags = soup.find_all(name="p")

#Separa en elementos de una lista las canciones.
songs_30s = tags[5].text.split('\n')

#Elimina los espacios de los extremos de cada elemento de la lista si
#los hubiese y devuelve una lista nueva con cada elemento
list_songs_30s = [i.strip() for i in songs_30s]

#Elimina los elementos vacios de la lista
list_songs_30s = list(filter(None, list_songs_30s))
#songs_30s = songs.split('\n')

list_songs_30s
tags[6]

In [ ]:
def list_songs_url(url):
    headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content)
    tags = soup.find_all(name="p")
    
    #Separa en elementos de una lista las canciones.
    songs = tags[5].text.split('\n')
    
    #Elimina los espacios de los extremos de cada elemento de la lista si
    #los hubiese y devuelve una lista nueva con cada elemento
    songs = [i.strip() for i in songs]
    
    #Elimina los elementos vacios de la lista
    songs = list(filter(None, songs))
    return songs

list_songs_url(url_dave)

In [ ]:
url_dave_70s = 'https://davesmusicdatabase.blogspot.com/2014/08/top-songs-from-1970-1979.html?m=0'

list_70s = list_songs_url(url_dave_70s)
list_70s

In [ ]:
test_str = '81. The Peanut Vendor (El Manisero)...Don Azpiazu with Antonio Machin (1930)'
test_str_70s = '1. “Stand by Me” Ben E. King (1961)'
test_url_90s = 'https://davesmusicdatabase.blogspot.com/2014/08/top-songs-from-1990-to-1999.html'

#Recibe un string y devuelve el a;o de el string de la cancion
def extract_year(song_string):
    return song_string[-5:-1]

extract_year(test_str)

In [ ]:
#Recibe un string y devuelve el orden de la cancion
def extract_order(song_string):
    return song_string[:3].replace('.','').strip()

extract_order(test_str)

In [ ]:
#Recibe un string y devuelve el titulo de la cancion
def extract_title(song_string):
    title_1 = re.findall('\d+.\s(.*)(?:\.\.\.|\…)',song_string)
    
    if title_1:
        return title_1[0]
    
    title_2 = re.findall('“(.*?)”', song_string)
    
    if title_2:
        return title_2[0]
    
    return ''


#test
for i in list_70s:
    song_70 = extract_title(i)
    print(song_70)

In [ ]:
#Recibe un string y devuelve el autor de la cancion
def extract_author(song_string):
    author_1 = re.findall('(?:\.\.\.|\…)(.*)\(',song_string)
    
    if author_1:
        return author_1[0]
    
    author_2 = re.findall('”\s(.*)\s\(',song_string)
    if author_2 == ' ':
        pass
    
    if author_2:
        return author_2[0]
    
    author_3 = re.findall('\.\s(.*)\s“',song_string)
    if author_3:
        return author_3[0]

#test
for i in list_70s:
    author_70 = extract_author(i)
    print(author_70)

In [ ]:
def extract_decade(song_string):
    return extract_year(song_string)[:3] + '0'

#test
extract_decade(test_str)

In [ ]:
def ext_data_songs(url_dave):
    list_songs_dave = list_songs_url(url_dave)
    formated_songs = []
    for i in list_songs_dave:
        decade = extract_decade(i)
        year = extract_year(i)
        title = extract_title(i)
        author = extract_author(i)
        order = extract_order(i)
        
        song = {
            'decade':decade,
            'year':year,
            'title':title,
            'author':author,
            'order':order,
        }
        formated_songs.append(song)
    return formated_songs

test = ext_data_songs(url_dave)
test

In [ ]:
url_dave_00s = 'https://davesmusicdatabase.blogspot.com/2017/10/the-top-100-songs-from-2000-2009.html'
url_dave_20s = 'https://davesmusicdatabase.blogspot.com/2014/07/top-100-songs-from-1920-1929.html'
url_dave_30s = 'https://davesmusicdatabase.blogspot.com/2014/07/top-100-songs-of-1930-1939.html'
url_dave_40s = 'https://davesmusicdatabase.blogspot.com/2014/07/top-100-songs-from-1940-1949.html'
url_dave_50s = 'https://davesmusicdatabase.blogspot.com/2014/07/top-100-songs-from-1950-to-1959.html'
url_dave_60s = 'https://davesmusicdatabase.blogspot.com/2014/08/top-100-songs-from-1960-1969_2.html'
url_dave_70s = 'https://davesmusicdatabase.blogspot.com/2014/08/top-songs-from-1970-1979.html?m=0'
url_dave_80s = 'https://davesmusicdatabase.blogspot.com/2014/08/top-100-songs-from-1980-to-1989.html'
url_dave_90s = 'https://davesmusicdatabase.blogspot.com/2014/08/top-songs-from-1990-to-1999.html'
url_dave_2010s = 'https://davesmusicdatabase.blogspot.com/2017/10/the-top-50-songs-from-2010-2016.html'

dicc_00s = ext_data_songs(url_dave_00s)
dicc_20s = ext_data_songs(url_dave_20s)
dicc_30s = ext_data_songs(url_dave_30s)
dicc_40s = ext_data_songs(url_dave_40s)
dicc_50s = ext_data_songs(url_dave_50s)
dicc_60s = ext_data_songs(url_dave_60s)
dicc_70s = ext_data_songs(url_dave_70s)
dicc_80s = ext_data_songs(url_dave_80s)
dicc_90s = ext_data_songs(url_dave_90s)
dicc_2010s = ext_data_songs(url_dave_2010s)

dicc_00s

In [ ]:
dicc_20s

In [ ]:
dicc_30s

In [ ]:
dicc_40s

In [ ]:
dicc_50s

In [ ]:
dicc_60s

In [ ]:
dicc_70s

In [ ]:
dicc_80s

In [ ]:
dicc_90s

In [ ]:
dicc_2010s

In [ ]:
#BuenosDias